In [25]:
import os
import pandas as pd
import soundfile as sf

In [26]:
BASE_PATH = os.getcwd()

In [27]:
data_path = os.path.join(BASE_PATH, 'data_aishell')

In [29]:
def read_text(file_path):
    with open(file_path, 'r') as f:
        text = f.readlines()
    return text

In [30]:
wav_list = []
set_list = []
path_list = []
duration_list = []
for data_set in os.listdir(os.path.join(data_path, 'wav')):
    for person in os.listdir(os.path.join(data_path, 'wav/{}'.format(data_set))):
        for wav in os.listdir(os.path.join(data_path, 'wav/{}/{}'.format(data_set, person))):
            wav_list.append(wav)
            set_list.append(data_set)
            path_list.append('{}/wav/{}/{}/{}'.format('aishell_1/data_aishell',data_set, person, wav))
            sig,sr = sf.read(os.path.join(data_path, 'wav/{}/{}/{}'.format(data_set,person, wav)))
            duration_list.append(round(float(len(sig))/float(sr), 3))

In [31]:
data = pd.DataFrame()

In [32]:
data["wav"] = wav_list
data["wav_path"] = path_list
data["durations"] = duration_list
data["data_set"] = set_list

In [33]:
data = data.sort_values(by='wav')

In [34]:
text = read_text(os.path.join(data_path, "transcript/aishell_transcript_v0.8.txt"))

In [35]:
wav_name_list = []
text_list = []
for i in sorted(text):
    wav_name = i[:16]
    wav_name_list.append(wav_name)
    txt = i[16:].strip('\n').replace(" ", '')
    text_list.append(txt)

In [36]:
data['txt'] = text_list[:len(data.wav)]

In [37]:
print('total num of wav is {}'.format(len(data.wav)))

total num of wav is 2172


In [38]:
print('total durations is {}h'.format(round(sum(data.durations)/3600.0, 3)))

total durations is 2.877h


In [39]:
data.to_csv('{}/aishell_1.csv'.format(BASE_PATH), index=False)